Need to create the mysql interpreter from the default jdbc Zeppelin interpreter.
---------------------------------------------------------------

Setup:  See the documentation for full up to date steps and up to date files.  Summarized below.  
---------
1) download the mysql java driver / connector / deb: https://dev.mysql.com/downloads/connector/j/
2) from terminal go to dir the file was downloaded and run - 
    - sudo dpkg -i mysql-connector-java_8.0.23-1ubuntu20.04_all.deb
      - result copied jar file to:     ./usr/share/java/mysql-connector-java-8.0.23.jar        -- mariadb-java-client.jar there as well 
3)sudo apt install libmariadb-java    
4)export CLASSPATH=/usr/share/java/mysql-connector-java-8.0.23.jar:$CLASSPATH
  export CLASSPATH=/usr/share/java/mariadb-java-client.jar.jar:$CLASSPATH
  
5)create mysql interpreter from jdbc in Zeppelin: 
   driver: com.mysql.jdbc.Driver    other options:  org.mariadb.jdbc.Driver    com.mysql.cj.jdbc.Driver
   URL:   jdbc:mysql://localhost:3306/galadrome  
     -- optumine is the default database.  If want to be able to use all databases on MYSQL then do not add optumine to the URL = jdbc:mysql://localhost:3306/
   default.user: galadrome_mysql
   defaul.password: Galadrome_mysql$9
   under dependencies artifact at the bottom enter:   mysql:mysql-connector-java:8.0.33 
 
      - need to manually add the default.password name at the bottom and in the value enter the users password
      
     
-------------------------------------------------------  -
  
examples: https://zeppelin.apache.org/docs/0.5.5-incubating/interpreter/postgresql.html

In [1]:
%pyspark
## MySQL options
mysqloptions = {
  "url" : "jdbc:mysql://localhost:3306/galadrome",
  "driver" : "com.mysql.jdbc.Driver",  
  "user" : "galadrome_mysql",
  "password" : "Galadrome_mysql$9", 
}
# import using query 
# test = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/").option("driver", "com.mysql.jdbc.Driver").option("query","select * from galadrome.training_auto limit 10").load()
test = sqlContext.read.format("jdbc").options(**mysqloptions).option("query","select RefID, IsBadBuy, PurchDate, Auction, VehYear, Make, Model from galadrome.training_auto limit 10").load()
test.show(5)
test.count()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.
+-----+--------+---------+-------+-------+-----+-------------------+
|RefID|IsBadBuy|PurchDate|Auction|VehYear| Make|              Model|
+-----+--------+---------+-------+-------+-----+-------------------+
|    1|       0|12/7/2009|  ADESA|   2006|MAZDA|             MAZDA3|
|    2|       0|12/7/2009|  ADESA|   2004|DODGE|1500 RAM PICKUP 2WD|
|    3|       0|12/7/2009|  ADESA|   2005|DODGE|         STRATUS V6|
|    4|       0|12/7/2009|  ADESA|   2004|DODGE|               NEON|
|    5|       0|12/7/2009|  ADESA|   2005| FORD|              FOCUS|
+-----+--------+---------+-------+-------+-----+-------------------+
only showing top 5 rows

10


In [2]:
%pyspark

test_df = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/galadrome") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "training_auto") \
    .option("user", "galadrome_mysql").option("password", "Galadrome_mysql$9").load()
test_df.show(5)

+-----+--------+---------+-------+-------+----------+-----+-------------------+----+-----------------+------+------------+-----------+---------+------+-----------+-----------+--------------------+---------------------------------+-------------------------------+--------------------------------+-----------------------------+-----------------------------+---------------------------+----------------------------+--------------------------+---------+--------+-----+------+----+--------+------------+------------+
|RefId|IsBadBuy|PurchDate|Auction|VehYear|VehicleAge| Make|              Model|Trim|         SubModel| Color|Transmission|WheelTypeID|WheelType|VehOdo|Nationality|       Size|TopThreeAmericanName|MMRAcquisitionAuctionAveragePrice|MMRAcquisitionAuctionCleanPrice|MMRAcquisitionRetailAveragePrice|MMRAcquisitonRetailCleanPrice|MMRCurrentAuctionAveragePrice|MMRCurrentAuctionCleanPrice|MMRCurrentRetailAveragePrice|MMRCurrentRetailCleanPrice|PRIMEUNIT|AUCGUART|BYRNO|VNZIP1|VNST|VehBCost|IsO

In [3]:
%pyspark

sqlContext.read.format("jdbc").options(**mysqloptions).option("query","Drop Table If EXISTS galadrome.test")
 ## drop table above does not work - need to research. Can use drop table in mysql process below.  research sqlcontext.query  
 
test.registerTempTable("testT")
sqlContext.sql("select RefID, IsBadBuy, PurchDate, Auction, VehYear, Make, Model from testT").write.format("jdbc").options(**mysqloptions).option("dbtable", "galadrome.test").mode("Overwrite").save()
    ## .mode('Overwrite')  OR     .mode("append").save()
# sqlContext.sql("CREATE TABLE galadrome.test AS SELECT RefID, IsBadBuy, PurchDate, Auction, VehYear, Make, Model from testT")

/opt/spark/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [4]:
%pyspark 
## from pyspark.sql import SparkSession

hostname="localhost"
dbname = "galadrome"
jdbcPort = 3306
username = "galadrome_mysql"
password = "Galadrome_mysql$9"
jdbc_url = "jdbc:mysql://{0}:{1}/{2}?user={3}&password={4}".format(hostname,jdbcPort, dbname,username,password)

query = "(select * from galadrome.training_auto limit 5) as table1"
df1 = spark.read.format('jdbc').options(driver = 'com.mysql.jdbc.Driver',url=jdbc_url, dbtable=query ).load()
df1.show()

+-----+--------+---------+-------+-------+----------+-----+-------------------+----+-----------------+------+------------+-----------+---------+------+-----------+-----------+--------------------+---------------------------------+-------------------------------+--------------------------------+-----------------------------+-----------------------------+---------------------------+----------------------------+--------------------------+---------+--------+-----+------+----+--------+------------+------------+
|RefId|IsBadBuy|PurchDate|Auction|VehYear|VehicleAge| Make|              Model|Trim|         SubModel| Color|Transmission|WheelTypeID|WheelType|VehOdo|Nationality|       Size|TopThreeAmericanName|MMRAcquisitionAuctionAveragePrice|MMRAcquisitionAuctionCleanPrice|MMRAcquisitionRetailAveragePrice|MMRAcquisitonRetailCleanPrice|MMRCurrentAuctionAveragePrice|MMRCurrentAuctionCleanPrice|MMRCurrentRetailAveragePrice|MMRCurrentRetailCleanPrice|PRIMEUNIT|AUCGUART|BYRNO|VNZIP1|VNST|VehBCost|IsO

In [5]:
%pyspark

## Mysql connection
import MySQLdb as mdb
import pandas.io.sql as sql
from pandas import *

## import from MySQL tables
 #conn = mdb.connect('server','user','password','schema');
conn = mdb.connect('localhost','galadrome_mysql','Galadrome_mysql$9','galadrome');  
%time visits = sql.read_frame('select * from galadrome.training_auto limit 10', conn)

In [6]:
# mysql
     import pymssql                                                        
     conn = pymssql.connect(host='localhost', user='galadrome_mysql', password='Galadrome_mysql$9', database='galadrome')
     data = sql.read_frame('select * from galadrome.training_auto limit 10', con)
     data
     
     # ODBC connections: See following for details - http://code.google.com/p/pyodbc/wiki/Objects
     import pyodbc # need to install first, see > http://code.google.com/p/pyodbc/wiki/Objects
     con = pyodbc.connect('DRIVER={MySQLDSN};SERVER=localhost;DATABASE=galadrome;UID=galadrome_mysql;PWD=Galadrome_mysql$9')
     data = sql.read_frame('select * from galadrome.training_auto limit 10', con)
     data
     data.ix[1] # select first row